In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()#.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-140023
Azure region: southcentralus
Subscription id: 610d6e37-4747-4a20-80eb-3aad70a55f43
Resource group: aml-quickstarts-140023


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_name = "cpu-cluster"

if (compute_name in ws.compute_targets) and (type(ws.compute_targets[compute_name]) is AmlCompute):
    compute_target = ws.compute_targets[compute_name]
    print("Using existing compute cluster: " + compute_name)
else:
    compute_config = AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    compute_target = ComputeTarget.create(ws, compute_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Using existing compute cluster: cpu-cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "C": uniform(0.5, 2.0),
        "max_iter": choice(range(50, 200))
    }
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
# (!) error msg says SKLearn module is deprecated and to use ScriptRunConfig instead
from azureml.core import ScriptRunConfig, Environment
env = Environment.get(ws, name="AzureML-Tutorial")
est = ScriptRunConfig(source_directory=".",
                      script="train.py",
                      compute_target=compute_target,
                      environment=env
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=est,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4
)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [10]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()

print("ID: ", best_run.id)
print(best_run_metrics)

#joblib.dump(value=best_run, filename="hypermodel.joblib")
#model = best_run.register_model(model_name="hyperbestmodel",
#                                model_path="outputs/hypermodel.joblib"
#)
### YOUR CODE HERE ###

ID:  HD_9ccbb790-b5ac-4cad-a2c8-2510bebb0bf3_4
{'Regularization Strength:': 1.7185040137631329, 'Max iterations:': 135, 'Accuracy': 0.9146637533381888}


In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [12]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###